In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp


In [9]:
subdirectory = 'merged_by_country_bert_processed_tweets'
merged_df = pd.DataFrame()

for filename in os.listdir(subdirectory):
    
    if filename.endswith('.csv'):
        file_path = os.path.join(subdirectory, filename)

        # Read the CSV file with UTF-8 encoding
        df = pd.read_csv(file_path, encoding='UTF-8', low_memory=False)

        merged_df = pd.concat([merged_df, df], ignore_index=True)

In [10]:
#BEGIN[ChatGPT][https://chat.openai.com/auth/login]

# I copy and pasted the ChatGPT to make a skeleton code and then I revised the code which did not work or weird

# command: We have a DataFrame (df) containing the Twitter users' stance on the Russia-Ukraine war. 
# The DataFrame has columns such as 'country' and 'sentiment' related to the war. 
# We want to identify the top 20 countries with the most users in the dataset and test the null hypothesis that people in these countries are not negatively biased towards the Russia-Ukraine war. 
# There will be a total of 20 null hypotheses.
# Also, in the 'sentiment' column, negative emotions are expressed as NEGATIVE and positive emotions as POSITIVE. Please convert negative sentiment to -1 and positive sentiment to 1

# explaination : This code analyzes the stance of Twitter users on the Russia-Ukraine war in a given DataFrame (df) containing columns like 'country' and 'sentiment'. 
# It identifies the top 20 countries with the most users in the dataset and tests the null hypothesis that people in these countries are not negatively biased towards the war.
# First, it converts the sentiments in the 'sentiment' column to -1 (negative) and 1 (positive). 
# Then, it calculates the number of users by country and selects the top 20 countries. 
# For each country, the code performs a null hypothesis test, calculates the negative sentiment ratio, and uses a t-test to verify the null hypothesis. 
# The results indicate whether users in each country are negative or not negative towards the war



# Convert sentiment values to -1 for 'NEGATIVE' and 1 for 'POSITIVE'

df['sentiment'] = df['sentiment'].map({'NEGATIVE': -1, 'POSITIVE': 1})

# Calculate the number of users by country and select the top 20 countries
top_20_countries = df['country'].value_counts().head(20).index.tolist()

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Country', 'Negative Sentiment Ratio', 't-statistic', 'p-value', 'Null Hypothesis'])

rejected_count = 0

# Perform null hypothesis testing for each country
for country in top_20_countries:
    country_df = df[df['country'] == country]
    negative_sentiment_ratio = country_df[country_df['sentiment'] == -1].shape[0] / country_df.shape[0]
    
    # Null hypothesis: The ratio of negative sentiment is less than 0.5 (not negative towards the war)
    # Alternative hypothesis: The ratio of negative sentiment is greater or equal to 0.5 (negative towards the war)
    t_statistic, p_value = ttest_1samp(country_df['sentiment'], 0.5)
    
    # Store results in the DataFrame
    result = pd.DataFrame({'Country': [country],
                           'Negative Sentiment Ratio': [round(negative_sentiment_ratio, 2)],
                           't-statistic': [round(t_statistic, 2)],
                           'p-value': [round(p_value, 2)],
                           'Null Hypothesis': ['Accept' if p_value >= 0.05 else 'Reject']})
    results_df = pd.concat([results_df, result], ignore_index=True)
    
    if p_value < 0.05:
        rejected_count += 1

# Display the results in table format
display(results_df)

# Calculate the ratio of countries that rejected null hypothesis
rejected_ratio = rejected_count / len(top_20_countries)
print(f"The ratio of countries rejecting null hypothesis: {rejected_ratio:.2f}")


#END[ChatGPT]


,Country,Negative Sentiment Ratio,t-statistic,p-value,Null Hypothesis
0,United States,0.74,-18.33,0.00,Reject
1,Ukraine,0.61,-10.20,0.00,Reject
2,Pakistan,0.04,12.16,0.00,Reject
3,India,0.63,-8.81,0.00,Reject
4,Japan,0.61,-6.42,0.00,Reject
5,Denmark,0.76,-9.27,0.00,Reject
6,United Kingdom,0.78,-8.73,0.00,Reject
7,France,0.72,-6.41,0.00,Reject
8,Italy,0.61,-3.05,0.01,Reject
9,Bulgaria,0.38,-0.96,0.36,Accept


The ratio of countries rejecting null hypothesis: 0.70
